In [5]:
import pandas as pd 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session



In [ ]:
sql = """
select station_name, icscode, lat, lng from tt_h.all_stations 
where lat < 53.39926695813 
and lng > -2.54 and 
in_sprawl = false
"""

df = pd.read_sql(sql, conn)

In [ ]:
# df["dest_pc"]= "OX145FP"
# df2 = df.copy()
# df2["dest_pc"] = "CB113QY"

# df21 = df.copy()
# df21["dest_pc"] = "CM25LB"

# df3 = pd.concat([df,df2,df3])
# num = len(df3)

In [ ]:
df3["id"] = range(len(df3))

In [ ]:
# Create new table of each station and drive time and distane 

# df3.to_sql("stations_drivetime", engine, schema="tt_h", if_exists="replace", index=False)

# sql = """

# ALTER TABLE tt_h.stations_drivetime  ADD PRIMARY KEY (id);

# ALTER TABLE tt_h.stations_drivetime ADD google_request text;
# UPDATE tt_h.stations_drivetime SET google_request = null;

# ALTER TABLE tt_h.stations_drivetime ADD google_response text;
# UPDATE tt_h.stations_drivetime SET google_response = null;

# ALTER TABLE tt_h.stations_drivetime ADD drive_minutes float;
# UPDATE tt_h.stations_drivetime SET drive_minutes = null;

# ALTER TABLE tt_h.stations_drivetime ADD drive_miles float;
# UPDATE tt_h.stations_drivetime SET drive_miles = null;


# """
# cursor.execute(sql)
# conn.commit()

In [12]:
# Then populate with drive times
sql = """
select id from tt_h.stations_drivetime where drive_minutes is null
"""
ids = pd.read_sql(sql, conn)
ids = [int(i) for i in ids["id"]]
len(ids)

727

In [13]:
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session
Stations = Automapped_Base.classes.stations_drivetime
from google import get_drive_info

counter = 0
for id in ids:
    station = session.query(Stations).filter(Stations.id == id).one()
    
    try:
        info = get_drive_info(origin_lat = station.lat , origin_lng = station.lng , dest_pc = station.dest_pc)
        station.google_request = info["request"]
        station.google_response = info["response"]
        station.drive_minutes = info["minutes"]
        station.drive_miles = info["miles"]
        session.add(station)
        session.commit()
    except Exception as e:
        print("error on {}".format(counter))
        break
    
    counter +=1
    print(counter)

error on 0


In [ ]:
sql = """
select *
from tt_h.stations_drivetime
"""
df = pd.read_sql(sql, conn)
df.sample(20)